In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from networks import *
from menet.menets_rev import MeNets

In [ ]:
#@title Load preprocessed & subsampled data (LOOCV)

ids =                    np.load('../mpii_dataset_shared/loocv_ids.npy')
images = torch.as_tensor(np.load('../mpii_dataset_shared/loocv_images.npy'), dtype=torch.float)
hps =    torch.as_tensor(np.load('../mpii_dataset_shared/loocv_2d_hps.npy'), dtype=torch.float)
gazes =  torch.as_tensor(np.load('../mpii_dataset_shared/loocv_2d_gazes.npy'), dtype=torch.float)

In [ ]:
device = torch.device('cuda:0')
seed = 10

experiment_name='mpii_loocv'
res_list = []

In [ ]:
for looid in range(15) : 

    train_ids = np.concatenate([ids[:looid], ids[(looid+1):]]).reshape(-1)
    train_images = torch.cat([images[:looid], images[(looid+1):]]).reshape(-1,36,60)
    train_hps = torch.cat([hps[:looid], hps[(looid+1):]]).reshape(-1,2)
    train_gazes = torch.cat([gazes[:looid], gazes[(looid+1):]]).reshape(-1,2)

    test_ids = ids[looid]
    test_images = images[looid]
    test_hps = hps[looid]
    test_gazes = gazes[looid]

    res_list.append(MeNets(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        ResNet.ResNet, max_iter=1, patience=1,
        test_unseen=True, result_path = '.', save=True,
        device=device, SEED=seed + looid, experiment_name = f'{experiment_name}_{looid}'
    ))

